In [1]:
import sys
import os
import json
import re

sys.path.append('../..')

import lcatools
from lcatools.foreground.manager import ForegroundManager
from lcatools.interact import *

In [2]:
F = ForegroundManager('.')

ForegroundArchive archive added in position 0
IlcdLcia archive added in position 1
JSON archive added in position 2
JSON archive added in position 3
JSON archive added in position 4
JSON archive added in position 5
JSON archive added in position 6
JSON archive added in position 7
JSON archive added in position 8
JSON archive added in position 9
JSON archive added in position 10
Generating flow-quantity database...
Loading LCIA data... (0.80 s)
Loading JSON data from /data/GitHub/lca-tools-datafiles/catalogs/elcd_lcia.json.gz:
Found Extension: zip
54 new quantity entities added (54 total)
1256 new flow entities added (1256 total)
0 new process entities added (0 total)
0 unmatched flows found from source LCIA... 

finished LCIA... (1.69 s)
Loading JSON data from /data/GitHub/lca-tools-datafiles/catalogs/ei_lcia.json.gz:
**Upstream reference encountered: /data/LCI/Ecoinvent/3.2/undefined

710 new quantity entities added (710 total)
3255 new flow entities added (3255 total)
0 new process e

In [3]:
frag = next(f for f in F[0].fragments(background=False))

In [4]:
F.draw_fragment(frag)

-<--*   4a37702 (      1) ethanol production from sugar beet molasses [RoW]
    | -<--B   bce6ce5 (      0) ammonium sulfate, as N
    | -<--B   0740a66 (      0) ethanol fermentation plant
    | =>=---: 2968b7b (      0) molasses, from sugar beet
    | -<--B   76a37f5 (   0.11) lime
    | -<--*   36ef048 (    3.8) sugar beet
    |  | -<--*   1799103 (      1) sugar beet
    |  |  | -<--B   8e1cb31 (5.1e-05) fertilising, by broadcaster
    |  |  | -<--B   3ba3c84 (5.7e-05) application of plant protection product, by field sprayer
    |  |  | -<--B   95ded38 (4.2e-05) potassium sulfate, as K2O
    |  |  | -<--B   cf3497a (0.00028) ammonium nitrate, as N
    |  |  | -<--B   e6d8b98 (2.6e-05) hoeing
    |  |  | -<--B   47c0ca2 (2.6e-05) triazine-compound, unspecified
    |  |  | -<--B   c330e5a (0.00017) liquid manure spreading, by vacuum tanker
    |  |  | -<--B   8070ae7 (2.7e-05) sugar beet seed, for sowing
    |  |  | -<--B   a925df0 (2.6e-05) fodder loading, by self-loading trailer
 

In [5]:
ffs = F.traverse(frag)

In [6]:
lcia_methods = (q for q in F[0].lcia_methods())

In [11]:
quantity = next(lcia_methods)
str(quantity)

'ReCiPe Midpoint (I), agricultural land occupation, ALOP [LCIA]'

In [12]:

total = 0.0
print('%s' % quantity)
skips = []
for i in ffs:
    mag = i.node_weight
    scr = i.term.score_cache(quantity)
    if scr is None:
        print('skipping this one: %s' % i.fragment['Name'])
        skips.append(i)
        continue
    cont = mag * scr.total()
    #if i.fragment.direction == i.term.direction:
    #    cont *= -1
    print('%10.3g x %10.3g = %10.3g %s' % (mag, scr.total(), cont, i.fragment['Name']))
    total += cont
print('Total: %10.6g' % total)

ReCiPe Midpoint (I), agricultural land occupation, ALOP [LCIA]
         1 x          0 =          0 ethanol production from sugar beet molasses [RoW]
         0 x      0.193 =          0 market for ammonium sulfate, as N [GLO]
         0 x   2.03e+05 =          0 market for ethanol fermentation plant [GLO]
skipping this one: molasses, from sugar beet
      0.11 x    0.00188 =   0.000207 market for lime [GLO]
      3.79 x          0 =          0 sugar beet
      3.79 x     0.0757 =      0.287 sugar beet
  0.000195 x       1.24 =   0.000241 market for fertilising, by broadcaster [GLO]
  0.000214 x       1.23 =   0.000263 market for application of plant protection product, by field sprayer [GLO]
  0.000159 x      0.324 =   5.16e-05 market for potassium sulfate, as K2O [GLO]
   0.00105 x       0.12 =   0.000126 market for ammonium nitrate, as N [GLO]
  9.74e-05 x       3.59 =    0.00035 market for hoeing [GLO]
  9.83e-05 x      0.443 =   4.35e-05 market for triazine-compound, unspecified [

In [13]:
F.bg_lcia(frag.term.term_node, [quantity], ref_flow=frag.term.term_flow, )[quantity.get_uuid()].show_details()

Loading LCIA results from f01eb904-d7e7-4633-b1a7-be54f33daa96_ffcb8cbb-77ab-4f87-8a94-87ca5b8ea79e.spold
ReCiPe Midpoint (I), agricultural land occupation, ALOP [LCIA] m2a
------------------------------------------------------------
         1 x      0.568 =      0.568 [RoW] ethanol, without water, in 95% solution state, from fermentation (CAS 000064-17-5) [By-product classification: allocatable product]
             Total score: 0.568269 
     0.568 ReCiPe Midpoint (I), agricultural land occupation, ALOP [LCIA]
